# Create Heat Map for significant DNA Replication genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [5]:
prot_enr.res2d.head(15)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,123/193,3.047452e-59,4.662602e-56,0,0,4.717289,635.599053,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,452/1631,6.421603e-57,4.912526e-54,0,0,2.051300,265.412919,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,92/134,8.907630e-49,4.542892e-46,0,0,5.081918,562.262229,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,92/144,1.076002e-44,4.115706e-42,0,0,4.729007,478.767028,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,124/348,5.502170e-26,1.683664e-23,0,0,2.637467,153.400560,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016
5,Transport of Mature Transcript to Cytoplasm Ho...,49/74,1.844730e-25,4.704062e-23,0,0,4.901274,279.138818,NUP107;NUP188;EIF4A3;MAGOH;NUP214;NDC1;NCBP1;N...,Reactome_2016
6,Major pathway of rRNA processing in the nucleo...,77/166,6.793384e-25,1.484840e-22,0,0,3.433423,191.065441,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;PWP2;RPL10A;R...,Reactome_2016
7,rRNA processing Homo sapiens R-HSA-72312,80/180,2.691962e-24,5.148376e-22,0,0,3.289744,178.540249,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;NAT10;PWP2;RP...,Reactome_2016
8,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",146/462,2.718785e-24,4.621935e-22,0,0,2.339136,126.925880,ANKLE2;NUP107;NUP188;ZWILCH;BUB1B;PPP2R2A;SMC4...,Reactome_2016
9,Transport of Mature mRNA derived from an Intro...,45/68,1.782404e-23,2.727077e-21,0,0,4.898332,256.581971,NUP205;NUP107;SEH1L;NUP188;RBM8A;EIF4A3;NUP160...,Reactome_2016


# Step 2: Get the list of significant genes in the cell cycle mitotic pathway

In [6]:
dna_rep = prot_enr.res2d.Genes[10]
genes = dna_rep.split(';')
print('total genes:',len(genes))

total genes: 165


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [7]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

165

In [11]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=500, plot_width= 2500)

Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [12]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [13]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'CENPM', 'LMNB1', 'MCM6', 'MCM4', 'GINS3', 'RFC5', 'MCM5',
       'MCM2', 'PCM1', 'MCM7', 'NUP37', 'MCM3', 'NUP153', 'SPC24', 'SMC4',
       'CDK11B', 'SMC2', 'SPC25', 'NCAPD2', 'RFC2', 'NUF2', 'RFC4',
       'TOPBP1', 'NCAPG', 'CHEK1', 'CENPK', 'CENPF', 'GINS4', 'NDC80',
       'POLA2', 'PPP2R2A', 'PPP1R12B', 'CDCA5', 'HDAC1', 'TPX2', 'NDEL1',
       'MIS18A', 'GINS2', 'BUB1B', 'TOP2A', 'RSF1', 'E2F5', 'RCC2',
       'DKC1', 'RRM2', 'NUP210', 'TOP3A', 'NEK9', 'PLK1', 'ODF2', 'CEP78',
       'GORASP1', 'SYNE1', 'PSMB8', 'TP53', 'PSMB9', 'RNF168'],
      dtype=object)

In [17]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1000)

Loading BokehJS ...